In [1]:
import tensorflow as tf

# Must come before any dataset/model creation
tf.config.run_functions_eagerly(True)
print("Eager execution:", tf.executing_eagerly())


Eager execution: True


In [2]:
import os

In [3]:
%pwd

'c:\\Users\\kanha\\Chicken_Disease_Classification_Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\kanha\\Chicken_Disease_Classification_Project'

In [18]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen = True)
class Trainingconfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size : int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float


@dataclass(frozen = True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path    



In [19]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [26]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_name])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
          config = self.config.prepare_callbacks
          model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
          create_directories([
                Path(model_ckpt_dir),
                Path(config.tensorboard_root_log_dir)
          ]) 

          prepare_callback_config = PrepareCallbackConfig(
                root_dir = Path(config.root_dir),
                tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
          )   
          
          return prepare_callback_config
    



    def get_training_config(self) -> Trainingconfig:
          training = self.config.training
          prepare_base_model = self.config.prepare_base_model
          params = self.params
          training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
          create_directories([
                Path(training.root_dir)
          ])

          training_config = Trainingconfig(
                root_dir = Path(training.root_dir),
                trained_model_path = Path(training.trained_model_path),
                updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
                training_data = Path(training_data),
                params_epochs = params.EPOCHS,
                params_batch_size = params.BATCH_SIZE,
                params_is_augmentation = params.AUGMENTATION,
                params_image_size = params.IMAGE_SIZE,
                params_learning_rate  = params.LEARNING_RATE

          )  

          return training_config

In [27]:
import time

In [28]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config  = config



    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_(timestamp)",

        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    



In [29]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [30]:
class Training:
    def __init__(self, config: Trainingconfig):
        self.config = config


    def get_base_model(self):

        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
   
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"

        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator  = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip = True,
                width_shift_range=0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs

            )
        else:
            train_datagenerator = valid_datagenerator 

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle=True,
            **dataflow_kwargs

        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )

        

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )    




In [31]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()



    training_config  = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list = callback_list
    )

except Exception as e:
    raise e

[2025-08-19 23:59:46,638: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-08-19 23:59:46,640: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-08-19 23:59:46,642: INFO: common: created directory at: artifacts]
[2025-08-19 23:59:46,644: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-08-19 23:59:46,646: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_root_log_dir]
[2025-08-19 23:59:46,650: INFO: common: created directory at: artifacts\training]
[2025-08-19 23:59:47,048: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\kanha\anaconda3\envs\Chickendisease\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\kanha\anaconda3\envs\Chickendisease\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.7205 - loss: 2.1740[2025-08-20 00:02:52,793: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
19/19 ━━━━━━━━━━━━━━━━━━━━ 185s 10s/step - accuracy: 0.7246 - loss: 2.1616 - val_accuracy: 0.9375 - val_loss: 1.7265
[2025-08-20 00:02:53,080: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


In [84]:
print(type(training.train_generator))
print(type(training.valid_generator))


<class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>
<class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>


In [85]:
print("train_generator type:", type(self.train_generator))
print("valid_generator type:", type(self.valid_generator))


NameError: name 'self' is not defined